## Load data

In [1]:
from lovit_textmining_dataset.navernews_10days import get_comments_paths
from soynlp.noun import LRNounExtractor_v2
from soynlp.tokenizer import LTokenizer
from soynlp.utils import DoublespaceLineCorpus

path = get_comments_paths(date='2016-10-20')
corpus = DoublespaceLineCorpus(path, iter_sent=True)

## Train Word Extractor and Tokenizer

명사 추출기를 이용하여 명사 점수를 단어 점수로 이용합니다. 뉴스 기사는 띄어쓰기가 잘 되어 있기 때문에 L-Tokenizer 를 이용하였습니다.

In [2]:
noun_extractor = LRNounExtractor_v2()
noun_scores = noun_extractor.train_extract(corpus)

word_scores = {noun:score.score for noun, score in noun_scores.items()}
tokenizer = LTokenizer(scores = word_scores)

tokenizer.tokenize('뉴스 기사를 이용하여 학습한 모델입니다', tolerance=0.3)

[Noun Extractor] use default predictors
[Noun Extractor] num features: pos=3929, neg=2321, common=107
[Noun Extractor] counting eojeols
[EojeolCounter] n eojeol = 695989 from 274369 sents. mem=0.200 Gb                    
[Noun Extractor] complete eojeol counter -> lr graph
[Noun Extractor] has been trained. #eojeols=2191206, mem=1.948 Gb
[Noun Extractor] batch prediction was completed for 167918 words
[Noun Extractor] checked compounds. discovered 146262 compounds
[Noun Extractor] postprocessing detaching_features : 151558 -> 119849
[Noun Extractor] postprocessing ignore_features : 119849 -> 119486
[Noun Extractor] postprocessing ignore_NJ : 119486 -> 117335
[Noun Extractor] 117335 nouns (146262 compounds) with min frequency=1
[Noun Extractor] flushing was done. mem=2.344 Gb                    
[Noun Extractor] 67.99 % eojeols are covered


['뉴스', '기사', '를', '이용', '하여', '학습', '한', '모델', '입니다']

## Build co-occurrence matrix

L parts 에서 명사 점수의 최대값과 0.3 점 차이가 나지 않는다면 그 중 가장 긴 subword 를 단어로 추출하도록 tolerance 를 이용하였습니다. min_tf 를 10 으로 설정하여 10 번 이하로 등장한 단어에 대해서는 co-occurrence 를 계산하지 않습니다.

In [3]:
from soynlp.vectorizer import sent_to_word_contexts_matrix

def custom_tokenizer(sent):
    return tokenizer.tokenize(sent, tolerance=0.3)

x, idx2vocab = sent_to_word_contexts_matrix(
    corpus,
    windows = 2,
    min_tf = 10,
    tokenizer = custom_tokenizer,
    verbose = True
)

Create (word, contexts) matrix
  - counting word frequency from 274368 sents, mem=2.533 Gb
  - scanning (word, context) pairs from 274368 sents, mem=2.533 Gb
  - (word, context) matrix was constructed. shape = (25954, 25954)                    
  - done


학습 후 30,810 개의 단어가 학습되었습니다.

In [4]:
x.shape

(25954, 25954)

## similar words using context vector

idx2vocab 을 vocab2idx 로 만듭니다. 

In [5]:
vocab2idx = {vocab:idx for idx, vocab in enumerate(idx2vocab)}

print(vocab2idx['이화여대'])
print(vocab2idx['아이오아이'])

1965
1780


sklearn.metrics.pairwise_distances 를 이용하여 '이화여대'와 context vector 가 비슷한 다른 단어를 찾습니다. 

numpy.ndarray 형식인 distance matrix (dist) 에 대하여 argsort() 를 하면, 거리 순서대로 정렬됩니다. sort() 를 하면 값이 정렬되며, argsort() 를 하면 각 값의 index 가 return 됩니다. 

여러 번 쓸 수 있도록 함수로도 만들어둡니다. 

In [6]:
import numpy as np
from sklearn.metrics import pairwise_distances

def most_similar_words(query, wv, topk=10):
    """
    query : str
    wv : word representation
    topk : num of similar terms
    """
    
    if not (query in vocab2idx):
        return []

    query_idx = vocab2idx[query]
    query_vec = wv[query_idx,:].reshape(1,-1)
    dist = pairwise_distances(query_vec, wv, metric='cosine')[0]
    similars = []

    # sorting
    for similar_idx in dist.argsort():
        
        # filtering query term
        if similar_idx == query_idx:
            continue

        if len(similars) >= topk:
            break

        # decoding index to word
        similar_word = idx2vocab[similar_idx]
        similars.append((similar_word, 1 - dist[similar_idx]))

    return similars

Cooccurrence matrix 만을 이용해도 문맥이 **매우** 확실한 단어 `아이오아이`는 다른 아이돌 이름이나 `아오아` 같은 약어가 유사어로 검색됩니다.

In [7]:
most_similar_words('아이오아이', x)

[('블랙핑크', 0.6755289243892053),
 ('트와이스', 0.622393548554468),
 ('엑소', 0.6033211536727949),
 ('우리나라', 0.5597736444192414),
 ('아오아', 0.5504612405602796),
 ('새누리', 0.5437366100639845),
 ('노래', 0.5421394745971377),
 ('우리', 0.541453581008619),
 ('에이핑크', 0.5350674956377075),
 ('김제동씨', 0.5314901989405921)]

그러나 단어 `아프리카`의 유사어는 잘 검색되지 않습니다. Co-occurrence frequency 를 representation 으로 이용하면 어느 문맥에나 등장하는 단어들에 영향을 받기 때문입니다.

In [8]:
most_similar_words('아프리카', x)

[('우리나라', 0.8037611170388616),
 ('회사', 0.8037363349622527),
 ('우리', 0.790296534084028),
 ('머리', 0.787885823985501),
 ('이나라', 0.7806776760312693),
 ('정부', 0.7785169302184154),
 ('스크린도어', 0.7652516200989101),
 ('여자', 0.7515296898487684),
 ('트럼프', 0.7500540890560078),
 ('새누리', 0.7490903619478668)]

## PPMI 를 이용한 term weighting

soynlp 의 pmi 를 이용하여 co-occurrence matrix 에 PMI 를 적용합니다. `min_pmi` 를 0 으로 설정하여 Positive PMI 로 만듭니다.

In [9]:
from soynlp.word import pmi

x_pmi, px, py = pmi(x, min_pmi=0.0)

PPMI 가 어느 문맥에나 등장하는 단어의 weight 를 0 으로 만들기 때문에 단어의 문맥이 뚜렷해집니다. `아이오아이`의 유사어는 크게 달라지지 않았습니다.

In [10]:
most_similar_words('아이오아이', x_pmi)

[('ioi', 0.1809293643175891),
 ('블랙핑크', 0.16269404370236584),
 ('IOI', 0.15126054480342965),
 ('블핑', 0.14611345956000332),
 ('트와이스', 0.1423420888369824),
 ('아오아', 0.14219738108886726),
 ('방탄', 0.1324156115896984),
 ('신곡', 0.12363733671446919),
 ('컴백', 0.11995624410107986),
 ('트와', 0.11677553503826488)]

그러나 `아프리카`의 유사어는 크게 바뀌었습니다. 다른 나라의 이름도 검색되며, `아프리카tv` 관련 단어인 `bj` 나 `유튜브`가 검색됩니다. 이날 대륙 `아프리카`에 관련된 기사와 `아프리카tv` 관련 기사가 모두 있었기 때문에 두 문맥이 모두 반영되어 있습니다.

그러나 representation vector 의 차원은 3 만 차원보다 큽니다.

In [11]:
most_similar_words('아프리카', x_pmi)

[('bj', 0.09174008479376838),
 ('비제이', 0.09034566575512504),
 ('유튜브', 0.08198154421369663),
 ('유투브', 0.07862526889424482),
 ('동남아', 0.07464494039211789),
 ('수수료', 0.07396822530612046),
 ('있을리', 0.0735855790226736),
 ('중동', 0.0733190010723227),
 ('삐지면', 0.069371423755756),
 ('남미', 0.06922054503695707)]

## PPMI + SVD 를 이용한 차원 축소

Singular Vector Decomposition (SVD) 를 이용하여 차원을 축소합니다.

In [12]:
from sklearn.decomposition import TruncatedSVD

svd = TruncatedSVD(n_components=300)
x_pmisvd = svd.fit_transform(x_pmi)

SVD 로 차원 축소를 하면 noise 가 어느 정도 제거됩니다. Co-occurrence matrix 에서의 noise 는 문맥이 특이하거나 infrequent 한 단어입니다. 그 외에는 PPMI 를 적용했을 때와 유사어의 관계는 비슷합니다. 하지만 cosine 유사도의 값은 커집니다. PPMI 만 적용하면 sparse vector 이기 때문에 평균적인 유사도가 작습니다. 하지만 SVD 를 적용하면 평균적인 유사도 scale 이 커집니다.

In [13]:
most_similar_words('아이오아이', x_pmisvd)

[('블랙핑크', 0.7981585963342795),
 ('트와이스', 0.7471412848682975),
 ('블핑', 0.7333447881893075),
 ('여자친구', 0.7070539576074238),
 ('ioi', 0.6885774490246201),
 ('아오아', 0.6850071208237152),
 ('트와', 0.68421616590016),
 ('컴백', 0.672958981872358),
 ('빅뱅', 0.662335156429146),
 ('신곡', 0.6614188063703947)]

In [14]:
most_similar_words('아프리카', x_pmisvd)

[('유튜브', 0.4449765493633768),
 ('인터넷', 0.43866986095898786),
 ('유투브', 0.43200091044265876),
 ('동남아', 0.4208089858095101),
 ('유럽', 0.4109185784018816),
 ('선진국', 0.4044908624512855),
 ('한국사람', 0.4040292067731064),
 ('국내', 0.4036200429166752),
 ('중동', 0.4007907540652521),
 ('TV', 0.39919460520603267)]

## S, Sigma, V 를 이용하여 representation 과 mapper 만들기

In [15]:
from sklearn.utils import check_random_state
from sklearn.utils.extmath import randomized_svd

def train(X, n_components, n_iter=5, random_state=None):

    if (random_state == None) or isinstance(random_state, int):
        random_state = check_random_state(random_state)

    n_features = X.shape[1]

    if n_components >= n_features:
        raise ValueError("n_components must be < n_features;"
                         " got %d >= %d" % (n_components, n_features))

    U, Sigma, VT = randomized_svd(
        X, n_components,
        n_iter = n_iter,
        random_state = random_state)

    S_ = Sigma ** (0.5)
    representation = y = U * S_
    mapper = (VT.T * S_).T

    return representation, mapper

wv, mapper = train(x_pmi, n_components=300)

In [16]:
most_similar_words('아이오아이', wv)

[('블랙핑크', 0.6963338694909909),
 ('트와이스', 0.6927600045816528),
 ('ioi', 0.6137406826522469),
 ('블핑', 0.6072816279453563),
 ('아오아', 0.5877446138325634),
 ('여자친구', 0.5770751000180948),
 ('신곡', 0.5692154765190212),
 ('트와', 0.5685233305841946),
 ('엑소', 0.5663855338125494),
 ('빅뱅', 0.5655052010733637)]

In [17]:
most_similar_words('아프리카', wv)

[('철구', 0.3300399080741523),
 ('비제이', 0.30671525295552393),
 ('동남아', 0.3005379534528194),
 ('유튜브', 0.3002160056698727),
 ('bj', 0.2994103509084116),
 ('티비', 0.27348955973975997),
 ('필리핀', 0.266659702734819),
 ('ㅅㅍ', 0.26646295692639677),
 ('중동', 0.2625211073841971),
 ('시청자', 0.25874905930033076)]